# 第二章 线程的安全性

## 线程安全性

什么是线程安全性？

>当多个线程访问某个类时，不管运行时的环境采用何种调度方式或者线程将如何交替执行，并且在主调代码中不需要任何额外的同步或者协同，这个类都能表现出正确的行为，那么就称这个类是线程安全的。

无状态的类都是**线程安全的**，这一点很容易理解，因为说一个类是线程不安全的无非就是说这个类在多线程并发访问下类的状态可能发生了非法变化：一个线程刚刚给该类的某个状态赋值，另一个线程立刻就改变了这个值导致刚才那个线程读到了不一致的数据。所谓线程安全就是要让线程和线程之间在访问某个对象时感知不到对方的存在。而如果一个对象是无状态的，那么刚好从根本上解决了这个问题。

## 竟态条件

出现线程安全问题的后果就是出现**竟态条件**，通俗的解释就是由于各个线程执行时机的不同而导致程序执行结果不一致。导致竟态条件的原因之一就是操作的**原子性**没有得到保证。事实上，并发编程的要解决的所有问题都可以归结到以下三个方面：

* 原子性：原子性即可以针对操作而言，也可以针对Java变量而言。对于操作而言就是一个操作是不可中断的，只有成功和失败两种状态。举例来说：自增运算不是原子性的，这是因为自增运算包含将变量读出，自增再写入三个子操作；而赋值操作就是原子性的，因为其只包含一个子操作就是赋值本身。原子变量是Java中一类特殊的变量，这种变量通过特殊的机器指令（例如`CAS`指令）将很多操作（例如自增运算）都原子化了。


* 可见性：由于Java的内存模型中分为工作内存和主存。因此在线程和线程之间共享变量时可能会出现变量可见性的问题，因为一个线程对变量的修改最先反映在其工作内存上，而其他线程要想读取到这个变量值的更改只有在这个变量的更改从工作内存反映到主存上之后才能实现，因此会出现可见性问题。解决这个问题通常可以通过`volatile`或者加锁来解决。使用`volatile`修饰变量是因为这样做能够让变量的更改马上显示在主存中从而解决这个问题，而对变量加锁操作也能让线程在释放锁的同时让变量的当前值写回主存中。


* 有序性：这个特性通常是线程内部指令重排问题的外在表现。导致这个问题的根源依然在于Java内存模型中工作内存以及主存的分别，当然，在线程内部不存在指令重排的问题，线程内部看待代码的执行都是顺序的，因为线程内部都是从工作内存中读取数据，而工作内存中的数据都是最新的，这叫做**线程内表现为串行语义**。但是在别的线程看来指令有可能出现重排，这还是由于普通对象更新后写入主存的时机不明确导致的，举例来说：某线程中为某两个变量赋值，赋值顺序存在偏序关系，那么在这个线程看来赋值的结果当然也是符合这种偏序关系的，因为更新工作内存的时机和赋值时机是一致的，因此一旦赋值就立即生效。但是这两个变量的改变从工作内存到主存的时机却是不确定的，很有可能赋值较晚的那个变量反而在工作内存中的值率先改变了，那么在另一个线程看来：这两个变量的复制顺序就是倒过来的，这就是指令重排。可以看出，指令重排在某种意义上说也是可见性的一种表现，因此也可以用`volatile`关键字和锁来解决。

## 加锁

加锁操作在Java中是必不可少的，虽然有很多**线程安全对象**，但是在复合操作环境下线程安全对象也并不是线程安全的，另一方面，加锁操作也能够保证变量的可见性。在Java中锁的知识点大致有以下几点：

* 加锁的对象：通常人们会认为：只有写入写对象的时候才需要加锁而读取对象则不需要，这种观点在大多数情况下是正确的，特别是对象状态不发生改变的状态下确实是这样的。但是由于共享对象可见性的问题，在读取对象的时候如果不加锁有可能会读取到**过时**的数据，因此在读取数据时有时候也需要加锁。


* 锁的重入性：所谓重入性就是说如果一个线程持有一个对象的锁，那么当其再次想要获取这个锁时无需阻塞就能立刻获取。锁的重入性通常的应用场景是：线程在子类同步方法中调用其父类的同名方法时，由于两个方法共享的是同一把锁，因此可以直接获取。另外就是在递归调用同步方法时也应用了锁的重入性。如果没有这样的特性可能会造成程序死锁。


* 程序的加锁后的性能：加锁可能会导致程序性能上的下降，但是这也是需要平衡的问题。

# 第三章 对象的共享

线程与线程之间共享对象的核心问题就是对象的：

1. 可见性。

2. 对象的线程安全性。

## 可见性

对象在线程之间的可见性问题的根源就是Java内存模型的设计问题。解决可见性问题的两个方式在上面一节中已经给出：

1. 使用`volatile`关键字修饰共享对象。（注，`volatile`关键字只保证变量的可见性但是并不保证其原子性）

2. 使用加锁技术来保证对象的可见性。

## 对象的线程安全性

### 封闭的（不共享）对象——绝对是线程安全的

首先需要明确的是如果一个对象不是共享的，那么这个对象将不存在安全性问题。一个对象要成为共享对象有两种方式：

1. 发布：这是程序员有意而为的，所谓发布就是将一个对象通过方法返回值，类字段等方式让其他的线程也能够访问得到。

2. 溢出：在效果上和发布是一致的，区别在溢出是程序员无意中导致的，变量溢出使得本不应该共享的对象变得共享，这可能会导致某些本身不是线程安全的对象被其他的线程使用从而造成线程安全问题，因此变量的溢出是需要检查和避免的。事实上，Java虚拟机中的**逃逸分析**就是专门做这个事儿的。

要想将一个变量封闭在某一个线程中，使其不在不经意间溢出，大概有以下三种方式可以采用：

1. `Ad-hoc`线程封闭：这种方式就是名字比较吓人，其实意思就是说把变量封闭在线程内部这件事完全由程序员自己去注意。因此这种封闭是非常脆弱的。

2. 栈封闭：这种封闭思路就是：将线程使用到的变量尽量转换成方法中的局部变量，因为Java的内存模型决定了线程方法存在于Java虚拟机栈中，而这个栈是线程隔离的，因此从根本上杜绝了变量逃逸的可能性。

3. `ThreadLocal`类：这个类是专门为线程存储数据而设计的，不用的线程使用相同的键值`key`调用`ThreadLocal.get`方法得到的结果是不一样的，这主要是应为这个对象会根据不同的线程ID将不同线程的数据隔离开，以避免变量溢出的问题。

### 线程安全的共享对象

共享对象就会存在线程安全问题，要解决这个问题可以由很多方法，通常可以有以下几种：

1. 使用不可变对象：不可变对象需要满足以下几点要求：
  1. 对象创建之后其状态就是不可修改的。
  2. 对象的所有域都是`final`类型的。
  3. 对象是正确创建的。
  
  满足这三个要求的才能称为不可变对象，在这里一定要区分的是：使用`final`修饰的对象不一定是不可变对象，因为可能这个这个对象它的子域字段并没有用`final`修饰，又可能这个对象它就是一个容器类，而使用`final`修饰的容器类是可以改变的，这种情况下`final`修饰的变量仅仅可以叫做**不可变引用**，而非不可变对象。当然，如果一个基本数据类型使用`final`修饰，那么可以肯定的说这个这个基本数据类型就是不可变的。如果一个对象是不可变的，那么这个对象一定是线程安全的。

2. 线程安全对象：Java中本身就自己带有很多线程安全对象，例如`ConcurrentHashMap`，`AtomicInteger`等等。这些对象原本就是线程安全的，因此也是可以安全共享的。

3. 加锁：这是不言而喻的。

### 安全发布对象

对象本身的特性保证线程安全之外，对象的发布手段也是保证其安全性的重要手段。想象一下，如果一个对象没能正确发布：比如说在其构造函数执行阶段被其他线程打断，导致发布了一个未能正确初始化的对象，这就会导致问题。安全发布的主要手段包含以下几种：

1. 在静态初始化函数中初始化一个对象引用：由于静态变量和静态代码块是在加载类阶段就执行的，因此这样做可以避免共享对象不正确初始化的问题。

2. 将对象引用保存在`volatile`中：这主要是为了保证可见性。

3. 将对象引用保存在某个正确构造的`final`域中：Java模型为不可变对象的共享提供一种特殊的初始化安全性保证。

4. 将对象的引用保存在一个由锁保护的域中。

# 第四章 对象的组合

>由`Servlet`容器实现的`ServletContext`对象必须是线程安全的，因为他肯定会被多个线程同时访问。当调用`setAttribute`和`getAttribute`时，`Servlet`不需要同步。

这一章主要是讲如何实现一个线程安全的类，其主要的思想是实现封闭，即一个线程安全的类中若有其他的线程不安全的类成员，则将其封闭在类内部这样就不会出现线程安全问题。若一定要访问这个线程不安全的类对象则返回其深拷贝对象或者给出一个不可变的成员对象。最后若是实在需要将成员对象发布，那么就将成员对象也做成线程安全的就可以了。

## 通过封闭来实现线程安全

这种思想就是要实现一个线程安全的类，首先需要将这个类的本身设计成线程安全的，有两种方式：一种是在每个方法前加上`synchronized`关键字，这样就能使类变成线程安全的。另一种方式是使用内置的锁变量，将整个类内部的临界区代码都用这同一个锁变量锁起来，这种方式也叫做**Java监视器模式**。

当这个类变成了线程安全的，但是类内部不可避免的会出现类成员变量。这些类成员变量有可能是线程不安全的，在这一小节中的解决方法就是采用封闭的方法。具体而言就是避免这些成员对象发布或溢出到类外部，如果外部一定需要访问这些类成员变量，那么就通过深拷贝的方式给出一个对象拷贝，这样就能避免线程安全问题。

## 通过委托来实现线程安全

所谓通过委托来实现线程安全就是将类线程安全的实现委托或者说寄希望于类成员变量的线程安全上。具体而言，如果一个类中有一个或者多个彼此**相互独立**且**没有值约束**（例如值被约束在某个区间范围内，或者一个成员变量的值依赖于另一个）的成员变量，这些成员变量都是线程安全的（例如这些成员变量都是**不可变**或者其本身就是**线程安全类**）。那么，这个包含线程安全成员对象的类不需要做额外的同步措施，其本身就是线程安全的。

## 增强现有的线程安全类

这是一种比较通用的办法，他的应用场景是：当需要增强现有的线程安全类时最好的办法是通过**组合**，**包装**，**继承**等手段增强这些类。具体的的例子如下：

1. 通过客户端加锁机制（其实是包装机制）增强原有`Collections.synchronizedList`线程安全类。

```Java
public class ListHelper<E>{
    public List<E> list = Collections.synchronizedList(new ArrayList<E>())
    ...
    public boolean putIfAbsent(E x){
        synchronized(this.list){
            boolean absent = !list.contains(x);
            if (absent){
                list.add(x);
            }
            return absent;
        }
    }
}
```

2. 通过组合机制增强原有类：

```Java
public class ImprovedList<T> implements List<T>{
    private final List<T> list;
    public ImprovedList(List<T> list){
        this.list = list
    }
    ...
   public synchronized boolean putIfAbsent(T x){
        boolean absent = !list.contains(x);
        if (absent){
            list.add(x);
        }
        return absent;
    }
    ...
}
```

3. 通过继承来增强原有线程安全类：

```Java
public class ImprovedVector<E> extends Vector<E>{
    public synchronized boolean putIfAbsent(E x){
        boolean absent = !contains(x);
        if (absent){
            add(x);
        }
        return absent;
    }
}
```

# 第五章 基础构架模块

线程安全的类也仅仅是**相对线程安全**，这就意味着在并发环境下线程安全类也绝对不是能任意使用的，仍然需要通过额外的同步措施来实现线程安全，这是因为很多操作是复合的，线程安全的实现需要通过这些复合操作的偏序执行才能保证，这也是加锁的目的所在。

## 线程安全容器对象在迭代时的问题及解决

例如`Vector`对象，其本身是一个线程安全类，但是如果并发地使用它，虽然它每个方法都是同步的，但是还是可能会出现数组越界的异常（这个错误的发生是由于在迭代的过程中有线程对容器进行了修改，导致出错），通常的解决方法就是在客户程序使用它时加额外的锁来保证其正确性。

如果通过迭代器`Iterator`或增强`for`循环（其实也是迭代）迭代一个线程安全的容器对象，就需要考虑在迭代过程中其他线程对该对象的修改问题，如果发生了这种情况，虚拟机会抛出一个`ConcurrentModificationException`异常，解决的方法是在迭代时对容器**对象加锁**或者迭代其深拷贝对象（极端不推荐）。

需要注意的是：类中有很多地方虽然没有显式地迭代对象，但是其实是在隐式地迭代对象，比如在`toString`方法中，`hashCode`方法，`equals`方法等等。在这些地方需要加锁同步，防止出现`ConcurrentModificationException`异常。

>虽然加锁可以防止迭代器抛出`ConcurrentModificationException`，但你必须记住**在对所有的共享容器进行迭代的地方都需要加锁**。

## 并发容器

这里介绍几种Java中的并发容器，也很常用。

### `ConcurrentHashMap`

这个类和`ConcurrentTable`非常相似，但是`ConcurrentHashMap`的并发性更高，这是因为与`ConcurrentTable`对整个容器对象加锁不同，`ConcurrentHashMap`加的是一种分段锁，这就意味着其并发性更高，同一时刻可以允许多个线程对其进行更改。而且在迭代的过程中如果修改了这个对象并不会抛出`ConcurrentModificationException`异常，代价就是`ConcurrentHashMap`的`isEmpty`，`size`等方法的返回值并不是精确的。当然，这个类增强有`putIfAbsent`等方法。

### `CopyOnWriteArrayList`

这是用于替代`List`，它也支持在迭代时修改而不抛出`ConcurrentModificationException`异常。所谓**“写入时复制”**指的是在修改时它会给出一个容器副本来供修改。

## 中断与阻塞

阻塞与很长的普通操作的区别在于：被阻塞的县城必须等待某个不受它控制的事件发生后才能继续执行，例如IO操作完成。

当线程发生中断时通常会抛出一个`InterruptedException`，这个异常的处理方法有两种：

* 传递`InterruptedException`：避开这个异常通常是最明智的策略——只需要将这个异常继续向上抛出，直到这个方法的调用者为止。

* 恢复中断：当无法继续抛出时，就处理这个异常，并且调用`Thread.currentThread().interrupt();`来中断（这样做的原因是在**阻塞方法**阻塞执行时发生中断，标准的操作是，抛出`InterruptedException()`并且清除中断标志位，也就是`Thread.isInterrupted()`将返回`false`，在处理异常模块中再次中断就是为了将中断标志位设置为`true`，即`Thread.isInterrupted()`为`true`）。

## 同步工具类

同步工具类的作用在于同步各个不同的线程，主要是有三种方式：

* 闭锁

* 信号量

* 栅栏

### 闭锁

>闭锁是一种同步工具类，可以延迟线程的进度直到其到达终止状态。闭锁的作用相当于一扇大门：在闭所到达结束状态之时，这扇门会打开并且允许所有的线程通过。党比所到达结束状态之后，将不会再改变状态，因此这扇门将永远保持开启状态。

在这里使用`CountDownLatch`类来作为例子，这个类就是实现闭锁的类，它含有一个阻塞方法`await()`，当线程调用这个方法时只要`CountDownLatch`的内部计数器没有减至0，这个方法就会一直阻塞线程。每当调用一次`countDown()`方法，`CountDownLatch`内部计数器就会减一，其举例如下：

```Java
@Test
public void test8() throws InterruptedException {
    	CountDownLatch latch = new CountDownLatch(1000);
    	for (int i = 0; i < 1000; i ++) {
    		Thread t = new Thread() {
    			public void run() {
    				try {
						this.sleep(1000);
					} catch (InterruptedException e) {
						// TODO Auto-generated catch block
						e.printStackTrace();
					}
    				latch.countDown();
    			}
    		};
    		t.start();
    	}
    	long start = System.currentTimeMillis();
    	latch.await();
    	System.out.println(System.currentTimeMillis() - start);
}
```

还有另一种闭锁，就是`FutureTask`，从名字上来看这是一种task，事实上，它是`Runnable`接口的一种实现。有以下三种状态：

* 等待运行。

* 正在运行。

* 运行完成。

这个类有一个方法`get`，当调用这个方法时如果任务已经完成，那么该方法就会立即返回否则将会阻塞直到任务完成。任务通过一个`Callable`接口的实现传入到`FutureTask`的构造方法中，举例如下：

```Java
public void test(){
    FutureTask<Object> task = new FutureTask<Object>(new Callable<Object>() {
			@Override
			public Object call() throws Exception {
				System.out.println("call() execuate");
				return new Object();
			}
    	}) {
    		public void run() {
    			System.out.println("run() execaute");
    			super.run();
    		}
    	};
    	Thread t1 = new Thread(task);
    	t1.start();
    	try {
			System.out.println(task.get());
		} catch (ExecutionException e) {
			e.printStackTrace();
		}
}
```

也可以这样写：

```Java
public void test10() {
    	FutureTask<Object> task = new FutureTask<Object>(new Callable<Object>() {
			@Override
			public Object call() throws Exception {
				System.out.println("call() execuate");
				return new Object();
			}
    	}) {
    		public void run() {
    			System.out.println("run() execaute");
    			super.run();
    		}
    	};
    	task.run();
        for (int i = 0; i < 1000; i ++) {
            Thread t = new Thread() {
                public void run() {
                    try {
						task.get();
					} catch (InterruptedException e) {
						// TODO Auto-generated catch block
						e.printStackTrace();
					} catch (ExecutionException e) {
						// TODO Auto-generated catch block
						e.printStackTrace();
					}
                }
            };
            t.start();
        }
    }
```

### 信号量

>计数信号量用来控制同时访问某个特定资源的操作数量，或者同时执行某个操作的数量。计数信号量还可以用来实现某种资源池，或者对容器加边界。

信号量`Semaphore`对象它有两个方法：`accuire()`和`release()`。如果信号量对象的计数器值大于0，那么`accuire()`将会直行通过且导致计数器值减一，如果一旦其内部计数器值为0，那么`accuire()`方法将会阻塞。当线程调用`release()`则信号量计数器的值会加一。以下是一个例子，用以限制集合内部元素的数量：

```Java
public class BoundedSet<T>{
    private final Set<T> set;
    private final Semaphore sem;
    
    public BoundedSet(int bound){
        this.set = Collections.synchronizedSet(new HashSet<T>());
        this.sem = new Semaphore(bound);
    }
    
    public boolean add(T e) throws InterruptedException{
        sem.accuire();
        boolean wasAdd = false;
        try{
            wasAdd = this.set.add(e);
            return wasAdd;
        }finally{
            if (!wasAdd){
                sem.release();
            }
        }
    }
    
    public boolean remove(T o){
        boolean wasRemoved = this.set.remove(o);
        if (wasRemoved){
            this.sem.release();
        }
        return wasRemoved;
    }
}
```

### 栅栏

>栅栏类似于闭锁，它能阻塞一组线程直到某个事件发生。栅栏与闭锁的关键区别在于：**所有线程必须同时到达栅栏的位置，才能继续执行。闭锁用于事件，而栅栏用于等待其他线程。**

使用举例如下：

```Java
public void test9() {
    	int threadCount = 2;
    	CyclicBarrier barrier = new CyclicBarrier(threadCount, new Runnable() {
			@Override
			public void run() {
				System.out.println("await completed - " + Thread.currentThread().getName());
			}
    	});
    	AtomicInteger count = new AtomicInteger(0);
    	for (int i = 0; i < threadCount; i ++) {
    		Thread t = new Thread() {
    			public void run() {
    				try {
    					int c = 0;
    					while((c ++) < 3) {
        					count.incrementAndGet();
            				System.out.println(c + " " + this.getName());
        					int index = barrier.await();
        					System.out.println(c + " " + this.getName() + " await() - " + index + " - " + count.get());
    					}
					} catch (InterruptedException e) {
						e.printStackTrace();
					} catch (BrokenBarrierException e) {
						e.printStackTrace();
					}
    			}
    		};
    		t.start();
    	}
    	try {
			Thread.currentThread().sleep(5000);
		} catch (InterruptedException e) {
			// TODO Auto-generated catch block
			e.printStackTrace();
		}
    }
```

输出如下：

```shell
1 Thread-0
1 Thread-1
await completed - Thread-1
1 Thread-1 await() - 0 - 2
2 Thread-1
1 Thread-0 await() - 1 - 3
2 Thread-0
await completed - Thread-0
2 Thread-0 await() - 0 - 4
3 Thread-0
2 Thread-1 await() - 1 - 4
3 Thread-1
await completed - Thread-1
3 Thread-1 await() - 0 - 6
3 Thread-0 await() - 1 - 6
```